# 모델 매개변수 최적화하기
- URL from : https://tutorials.pytorch.kr/beginner/basics/optimization_tutorial.html
- 모델을 학습하는 과정은 반복을 거친다. epoch이라고 부르는 각 반복 단계에서 모델은 출력값을 추측하고, 추측한 값과 정답  사이의 오류(손실=loss)를 계산하고, 매개변수에 대한 오류의 도함수(derivative)를 수집한 뒤, 경사하강법을 사용해서 이 파라미터들을 최적화(optimize)한다.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module) :
    def __init__(self) :
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x) :
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

하이퍼파라미터(Hyperparameter)
- 하이퍼파라미터는 모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수를 말한다. 서로 다른 하이퍼파라미터 값은 모델 학습과 수렴률(convergence rate)에 영향을 미칠 수 있다.
- 학습 시에는 다음과 같은 하이퍼파라미터를 정의한다.
    - epoch : 데이터셋을 반복하는 횟수
    - batch size : 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수
    - learning rate : 각 batch/epoch에서 모델의 매개변수를 조절하는 비율로, 값이 작을수록 학습 속도가 느려지고, 값이 크면 학습 중 예측할 수 없는 동작이 발생할 수 있다.

In [2]:
epochs = 5
batch_size = 128
learning_rate = 1e-3

최적화 단계(Optimization Loop)
- 하이퍼파라미터를 설정한 뒤에는 최적화 단계를 통해 모델을 학습하고 최적화할 수 있다. 최적화 단계의 각 반복(iteration)을 epoch이라고 부른다.
- 하나의 epoch은 다음의 두 부분으로 구성된다.
    - 학습 단계(train loop) : 학습용 데이터셋을 반복(iterate)하고 최적의 매개변수로 수렴한다.
    - 검증/시험 단계(validation/test loop) : 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복(iterate)한다.

손실 함수(Loss Function)
- 학습용 데이터를 제공하면, 학습되지 않은 신경망은 정답을 제공하지 않을 확률이 높다.
- 손실 함수는 획득한 결과와 실제 값 사이의 틀린 정도(degree of dissimilarity)를 측정하며, 학습 중에 이 값을 최소화하려고 한다. 주어진 데이터 샘플을 입력으로 계산한 예측과 정답(label)을 비교해서 손실(loss)를 계산한다.
- 일반적인 손실함수에는 회귀 문제(regression task)에 사용하는 nn.MSELoss(평균제곱오차, mean square error)나 분류(classification) 문제에 사용하는 nn.NLLLoss(음의 로그 우도, negative log likelihood), 그리고 nn.LogSoftmax와 nn.NLLLoss를 합친 nn.CrossEntropyLoss 등이 있다.
- 모델의 출력 로짓(logit)을 nn.CrossEntropyLoss에 전달하여 로짓(logit)을 정규화하고 예측 오류를 계산해보자

In [3]:
# 손실 함수를 초기화한다.
loss_fn = nn.CrossEntropyLoss()

옵티마이저(optimizer)
- 최적화는 각 학습 단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정이다. 최적화 알고리즘은 이 과정이 수행되는 방식을 정의하는데, 본 문서에서는 확률적 경사하강법(SGD, stochastic gradient descent)을 사용할 것이다.
- 모든 최적화 절차(logic)는 optimizer 객체에 캡슐화(encapsulate)된다.
- 학습하려는 모델의 매개변수와 학습률 하이퍼파라미터를 등록해서 옵티마이저를 초기화한다.

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

학습 단계에서 최적화는 3단계로 이루어진다.
- optimizer.zero_grad()를 호출해서 모델 매개변수의 변화도를 재설정한다. 기본적으로 변화도는 더해지기 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정해야 한다.
- loss.backwards()를 호출해서 예측 손실(prediction loss)를 역전파한다. 각 매개변수에 대한 손실의 변화도를 저장한다.
- 변화도를 계산한 뒤에는 optimizer.step()을 호출해서 역전파 단계에서 수집된 변화도로 매개변수를 조정한다.

최적화 코드를 반복하여 수행하는 train_loop와 테스트 데이터로 모델의 성능을 측정하는 test_loop를 정의한다.

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

손실 함수와 옵티마이저를 초기화하고 train_loop와 test_loop에 전달한다. 모델 성능 향상을 알아보기 위해 epoch수를 증가시켜볼 수도 있다.

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("끝!")

Epoch 1
-------------------------------
loss: 2.317406  [    0/60000]
loss: 2.298738  [ 6400/60000]
loss: 2.281718  [12800/60000]
loss: 2.271687  [19200/60000]
loss: 2.253535  [25600/60000]
loss: 2.232462  [32000/60000]
loss: 2.230743  [38400/60000]
loss: 2.203840  [44800/60000]
loss: 2.192293  [51200/60000]
loss: 2.159283  [57600/60000]
Test Error: 
 Accuracy: 46.9%, Avg loss: 2.154099 

Epoch 2
-------------------------------
loss: 2.164813  [    0/60000]
loss: 2.153424  [ 6400/60000]
loss: 2.094896  [12800/60000]
loss: 2.109108  [19200/60000]
loss: 2.051827  [25600/60000]
loss: 2.001537  [32000/60000]
loss: 2.017598  [38400/60000]
loss: 1.941753  [44800/60000]
loss: 1.937345  [51200/60000]
loss: 1.866813  [57600/60000]
Test Error: 
 Accuracy: 57.0%, Avg loss: 1.868035 

Epoch 3
-------------------------------
loss: 1.898900  [    0/60000]
loss: 1.871711  [ 6400/60000]
loss: 1.748349  [12800/60000]
loss: 1.793419  [19200/60000]
loss: 1.683150  [25600/60000]
loss: 1.644498  [32000/600